# AES Šifrovanje i Dešifrovanje Fajla u Pythonu

# Problem
Imamo fajl koji sadrži povjerljive informacije, i želimo da ga šifrujemo tako da se može dešifrovati samo pomoću tajnog ključa. Koristimo AES (Advanced Encryption Standard),
simetrični algoritam za enkripciju koji zahtjeva da oba korisnika imaju isti ključ.

# Ciljevi:
# - Šifrovati fajl `secret.txt` koristeći AES
# - Sačuvati šifrovani fajl kao `secret.txt.enc`
# - Dešifrovati i uporediti sa originalnim fajlom

In [2]:
%pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


    torch (>=1.8.*)
           ~~~~~~^

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 1. Kreiranje test fajla koji želimo da šifrujemo
with open("secret.txt", "w") as f:
    f.write("lozinka: MojSuperTajniPassword123\nPIN: 1234")
    
print("Fajl 'secret.txt' kreiran.")

Fajl 'secret.txt' kreiran.


In [1]:
# 2. Uvoz biblioteka
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

In [2]:
def encrypt_file(input_file, output_file, key):
    # Učitavamo sadržaj fajla
    with open(input_file, "rb") as f:
        data = f.read()
    
    # Generišemo slučajan IV (Initialization Vector)
    iv = get_random_bytes(16)

    # Inicijalizujemo AES u CBC režimu sa datim ključem i IV
    cipher = AES.new(key, AES.MODE_CBC, iv)
    
    # Šifrujemo podatke (potrebno ih je prethodno "napakovati")
    ciphertext = cipher.encrypt(pad(data, AES.block_size))

    # Čuvamo IV zajedno sa šifrovanim sadržajem
    with open(output_file, "wb") as f:
        f.write(iv + ciphertext)
    
    print(f"Fajl '{input_file}' je šifrovan i sačuvan kao '{output_file}'.")

In [3]:
# 4. Funkcija za dešifrovanje AES šifrovanog fajla

def decrypt_file(input_file, output_file, key):
    # Učitavamo IV i šifrovani sadržaj
    with open(input_file, "rb") as f:
        iv = f.read(16)
        ciphertext = f.read()

    # Inicijalizujemo AES za dešifrovanje
    cipher = AES.new(key, AES.MODE_CBC, iv)

    # Dešifrujemo i uklanjamo padding
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)

    # Sačuvamo rezultat
    with open(output_file, "wb") as f:
        f.write(plaintext)

    print(f"Fajl '{input_file}' je dešifrovan i sačuvan kao '{output_file}'.")


In [4]:
key = get_random_bytes(32)
print(f"AES ključ generisan ({len(key)*8}-bitni): {key.hex()}")

AES ključ generisan (256-bitni): c3dbb4c87c8e1bd43d42707b50b0e2c614c002768742afbf8c31f4ea6e39cc21


In [5]:
encrypt_file("secret.txt", "secret.txt.enc", key)

Fajl 'secret.txt' je šifrovan i sačuvan kao 'secret.txt.enc'.


In [6]:
decrypt_file("secret.txt.enc", "secret_decrypted.txt", key)

Fajl 'secret.txt.enc' je dešifrovan i sačuvan kao 'secret_decrypted.txt'.


In [7]:
with open("secret.txt", "rb") as original, open("secret_decrypted.txt", "rb") as decrypted:
    original_data = original.read()
    decrypted_data = decrypted.read()

    if original_data == decrypted_data:
        print("Fajl uspešno dešifrovan - identičan je originalu.")
    else:
        print("Greška: Fajl nakon dešifrovanja NIJE identičan originalu.")

Fajl uspešno dešifrovan - identičan je originalu.


AES (CBC mod) zahtjeva da se podaci dijele u blokove od 16 bajtova. Zato koristimo pad() i unpad() da obezbjedimo ispravnu veličinu.

IV (Initialization Vector) mora biti nasumičan i jedinstven za svaki fajl, ali nije tajan. Zato ga upisujemo u fajl prije šifrovanih podataka.

Tajni ključ (key) mora se čuvati bezbjedno. Ko god ima taj ključ, može dešifrovati fajl.